<a href="https://colab.research.google.com/github/google-research/protein-ligand-binding-free-energy-calculations/blob/matteo-dev/colab_tutorial/gromacs_abfe_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2022 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Dependencies and Mini Gromacs Tutorial

## Note

1. If you use a public colab instance, remember to change your runtime type to GPU
2. If you deployed a custom GCE runtime, please connect to a GPU VM instance.

## Summary

1. Pull Gromacs 2022.3 binary compiled specifically for Colab GPU environment and install the software.
2. Install Gromacs Python API on the fly, together with other packages used in the tutorial.
3. Run simulations for benchmarks and show how to use the official GMX
Python API to modify inputs.
4. Visualize the output conformation for the longer simulation.

In [1]:
#@title Install Gromacs and Gromacs Python API
!wget https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz -O /tmp/gromacs.tar.gz
!wget https://storage.googleapis.com/gromacs-bin/benchMEM.tpr -O /tmp/benchMEM.tpr
!wget https://storage.googleapis.com/gromacs-bin/hif2a_eq.tpr -O /tmp/hif2a_eq.tpr

!tar zxf /tmp/gromacs.tar.gz
!sudo rm -rf /usr/local/gromacs
!sudo mv gromacs-avx /usr/local/gromacs
!rm /tmp/gromacs.tar.gz

!pip3 install --upgrade pip
!pip3 install setuptools wheel cmake pybind11 py3DMol
# This ensure we always install the latest PMX version from GitHub.
!if [ -d /usr/local/lib/python3.7/dist-packages/pmx/ ]; then yes Y | pip3 uninstall pmx; fi
!pip3 install git+https://github.com/deGrootLab/pmx.git@abfe_dev
!gmxapi_ROOT=/usr/local/gromacs/ pip3 install --no-cache-dir gmxapi

--2022-11-08 21:22:00--  https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23920923 (23M) [application/x-gzip]
Saving to: ‘/tmp/gromacs.tar.gz’

/tmp/gromacs.tar.gz 100%[===================>]  22.81M  18.7MB/s    in 1.2s    

2022-11-08 21:22:01 (18.7 MB/s) - ‘/tmp/gromacs.tar.gz’ saved [23920923/23920923]

--2022-11-08 21:22:02--  https://storage.googleapis.com/gromacs-bin/benchMEM.tpr
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3461380 (3.3M) [application/octet-stream]
Savin

In [3]:
#@title Set Level for Logging
#@markdown Sets the python logging level. This is useful for debugging.
import logging

log_level = 'INFO' #@param ['DEBUG', 'INFO', 'ERROR']

_log_level = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'ERROR': logging.ERROR
}

logging.basicConfig(level=_log_level[log_level])

In [2]:
#@title [Please Rerun after Restart] Create a Session
#@markdown Please rerun this cell every time when you restart the runtime.
#@markdown A new session ID will be created for a new colab runtime session.
#@markdown Run this cell again will refresh the session ID.

import os
import time
import gmxapi


# Sets a session ID for this session.
_SESSION_ID = str(int(time.time())) #@param

# Monkey patch this session ID into the gmxapi.operation.ResourceManager.
# Hack. Do not use in the user code.
# This is temporarily added to allow users to start from a new set of working
# directories for all the mdrun launches. GMX API doesn't expose the
# underlying resource manager or context yet.
def operation_id(self):
  return f"{self._base_operation_id}_{_SESSION_ID}"
gmxapi.operation.ResourceManager.operation_id = property(operation_id)


def get_work_dir(md_output):
  values = [v for v in md_output.values()]
  return values[0].result()


def tail_log(md_output, output_file_base, last_n_lines=6):
  work_dir = get_work_dir(md_output)

  with open(os.path.join(work_dir, output_file_base + '.log'), 'r') as log:
    for line in log.readlines()[-last_n_lines:]:
      print(line)

print("Session ID: ", _SESSION_ID)

Session ID:  1667942570


In [ ]:
#@title Run a Short Simulation for Benchmark

# Read a TPR sample and run it.
# The TPR file configures a 20ps simulation.
input_tpr = gmxapi.read_tpr('/tmp/benchMEM.tpr')

# Can specify computations to GPU.
output_file_base = 'short'
md = gmxapi.mdrun(input_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})

# Run the simulation
md.run()

# Tail performance
tail_log(md.output, output_file_base)

INFO:gmxapi.mdrun:Building mdrun operation from source <gmxapi.simulation.read_tpr.StandardOperationHandle object at 0x7f156fb49710>
INFO:gmxapi.mdrun:mdrun receiving input _simulation_input: ResultDescription(dtype=<class 'str'>, width=1)
INFO:gmxapi.mdrun:mdrun input bound as source collection DataSourceCollection([('_simulation_input', <Future: name='_simulation_input', description=ResultDescription(dtype=<class 'str'>, width=1)>), ('parameters', <Future: name='parameters', description=ResultDescription(dtype=<class 'dict'>, width=1)>), ('runtime_args', {'-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu', '-deffnm': 'short'})])
INFO:gmxapi.simulation.context:Importing gmxapi.simulation.context
INFO:gmxapi.simulation.context:mpi4py is not available for default session communication.
INFO:gmxapi.mdrun:Created /content/mdrun_0_1667586554_0/topol.tpr on rank 0
INFO:gmxapi.simulation.workflow:Added element md_sim to workspec.
INFO:gmxapi.simulation.context:Collected builder 

               Core t (s)   Wall t (s)        (%)

       Time:      168.602       42.151      400.0

                 (ns/day)    (hour/ns)

Performance:       41.000        0.585

Finished mdrun on rank 0 Fri Nov  4 18:30:00 2022





In [ ]:
#@title Modified Params and Run A Longer Simulation
# Run the simulation for 40 ps.
output_file_base = 'long2'

modified_tpr = gmxapi.modify_input(
    input=input_tpr, parameters={'nsteps': 20000, 'nstlist': 100})

md = gmxapi.mdrun(modified_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})
md.run()

# Tail performance
tail_log(md.output, output_file_base)

In [ ]:
#@title What about a binding affinity simulation with HIF2a?
output_file_base = 'hif'

input_tpr = gmx.read_tpr('/tmp/hif2a_eq.tpr')
modified_tpr = gmx.modify_input(
    input=input_tpr, parameters={'nsteps': 20000, 'nstlist': 100})
md = gmx.mdrun(modified_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})
md.run()

# Tail performance
tail_log(md.output, output_file_base)

## Visualization

Visualize the output from the above HIF2A simulation.
We use py3DMol to visualize the output gro file.

In [ ]:
#@title Visualize the output conformation of the HIF2A simulation.
import py3Dmol

view = py3Dmol.view()
work_dir = get_work_dir(md.output)

view.addModel(
    open(os.path.join(work_dir, output_file_base + '.gro'), 'r').read(), 'gro')

view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({}, {'cartoon': {'color': 'spectrum'}})

view.show()

# ABFE Workflow
-----

In [4]:
import numpy as np
import shutil
from tqdm import tqdm
import pmx
from pmx.AbsoluteDG import AbsoluteDG

RDKit imports failed


In [100]:
#@title Define some helper functions

def mdrun(tpr, pmegpu=True, nsteps=None, transition=False):
  """Wrapper for gmxapi.mdrun with predefined scenarios optimized for different 
  types of simulations.

  Args:
    tpr (str): path to TPR file.
    pmegpu (bool): whether to run PME calculations on the GPU. Default is True.
      Note this is not possible with certain integrators.
    nsteps (int): number of integration steps to run, overwrites nsteps defined
      by the MDP file used to generate the TPR. Default is None (do not modify 
      TPR).
    transition (bool): whether we are running a non-equilibrium transition. 
      Default is False.

  Returns:
    object: StandardOperationHandle returned by gmxapi.mdrun.
  """
  # Load TPR file.
  input_tpr = gmxapi.read_tpr(tpr)
  if nsteps is not None:
    input_tpr = gmxapi.modify_input(input=input_tpr, 
                                    parameters={'nsteps': nsteps})    

  # Get path to TPR.
  path = "/".join(tpr.split('/')[:-1])
  
  # Specify command line flags and arguments to be passed to mdrun.
  # Note: energy minimization cannot run PME on GPU.
  if pmegpu:
    _pme = 'gpu'
    _pmefft = 'gpu'
    _bonded = 'gpu'
  else:
    _pme = 'auto'
    _pmefft = 'auto'
    _bonded = 'auto'

  # If we are running a non-equilibrium transition, we want to get the dhdl
  # file and keep only that. We then use the presence of dhdl_{n}.xvg to 
  # determine if the simulation has been ran already.
  if transition:
    n = int(tpr.split('/')[-1].split('.')[0].replace('ti', ''))
    md = gmxapi.mdrun(input_tpr,
                      runtime_args={'-nb': 'gpu', 
                                    '-pme': _pme, 
                                    '-pmefft': _pmefft, 
                                    '-bonded': _bonded,
                                    '-x': f'{path}/traj_{n}.xtc',
                                    '-o': f'{path}/traj_{n}.trr',
                                    '-c': f'{path}/confout_{n}.gro',
                                    '-e': f'{path}/ener_{n}.edr', 
                                    '-g': f'{path}/md_{n}.log',
                                    '-cpo': f'{path}/state_{n}.cpt',
                                    '-dhdl': f'{path}/dhdl_{n}.xvg'
                                    }
                      )
  else:
    md = gmxapi.mdrun(input_tpr,
                      runtime_args={'-nb': 'gpu', 
                                    '-pme': _pme, 
                                    '-pmefft': _pmefft, 
                                    '-bonded': _bonded,
                                    '-x': f'{path}/traj.xtc',
                                    '-o': f'{path}/traj.trr',
                                    '-c': f'{path}/confout.gro',
                                    '-e': f'{path}/ener.edr', 
                                    '-g': f'{path}/md.log',
                                    '-cpo': f'{path}/state.cpt'
                                    }
                      )

  # Run the simulation.
  md.run()

  # If transition, clean up files we won't need.
  if transition:
    for f in [f'{path}/traj_{n}.xtc', 
              f'{path}/traj_{n}.trr', 
              f'{path}/confout_{n}.gro', 
              f'{path}/ener_{n}.edr', 
              f'{path}/md_{n}.log',  
              f'{path}/state_{n}.cpt']:
      if os.path.isfile(f):
        os.remove(f)

  return md


def mdrun_completed(tpr: str, transition: bool = False) -> bool:
  """Checks whether energy minimization completed successfully.
  
  Args:
    tpr (str): path to TPR file.
    transition (bool): whether we are checking completion for a non-equilibrium 
      transition. Default is False.

  Returns:
    bool: whether the TPR has been run successfully.
  """
  # If we're chekcing the completion of a non-eq transition, we check that (i)
  # the right dhdl.xvg file exists, and that (ii) it contains info up to the 
  # end of the transition. Otherwise, we assume it crashed and needs to be
  # completed.
  if transition:
    # Get the index of the transition.
    n = int(tpr.split('/')[-1].split('.')[0].replace('ti', ''))
    # Get dhdl file path.
    dhdl = "/".join(tpr.split("/")[:-1]) + f"/dhdl_{n}.xvg"
    # If dhdl file exists, check it's complete.
    if os.path.isfile(dhdl):
      input_tpr = gmxapi.read_tpr(tpr)
      nsteps = input_tpr.output.parameters.result()['nsteps']
      dt = input_tpr.output.parameters.result()['dt']
      expected_final_time = nsteps * dt
      
      with open(dhdl, 'r') as f:
        lines = f.readlines()
      
      try:
        actual_final_time = float(lines[-1].split()[0])
        if expected_final_time - actual_final_time < 1e-6:
          return True
      except:
        return False
      
    return False
  else:
    # (maldeghi): IIRC Gromacs would output the GRO file only for mdruns that
    # did not crash/errored. We can make this stricted by checking the log
    # file too.
    gro = "/".join(tpr.split("/")[:-1]) + "/confout.gro"
    if os.path.isfile(gro):
      return True
    else:
      return False


def tail(fname, n):
  with open(fname) as f:
    for line in (f.readlines() [-n:]):
      print(line, end ='')

In [6]:
#@title Copy input files

#_WORKDIR = f"/content/pmxrun_{_SESSION_ID}"
_WORKDIR = f"/content/pmxrun_matteo"

if not os.path.isdir(_WORKDIR):
  os.mkdir(_WORKDIR)

  # Copy topology and structure files for protein and ligand.
  shutil.copytree('/usr/local/lib/python3.7/dist-packages/pmx/abfe_scripts/struct_top/', f'{_WORKDIR}/struct_top/')

  # Copy MDP files (i.e. Gromacs config files).
  shutil.copytree('/usr/local/lib/python3.7/dist-packages/pmx/abfe_scripts/mdppath/', f'{_WORKDIR}/mdppath/')

# Quick fix to the MDP files: use pme-order 4 becuase this is the only order
# supported to be able to run PME on the GPU.
! sed -i 's/pme-order.*/pme-order = 4/g' $_WORKDIR/mdppath/*

# We also shorten the alchemical transitions to speed things up.
# Note we cannot just change nsteps in mdrun here because we need to adjust the
# lambda schedule too.
! sed -i 's/nsteps.*/nsteps = 25000/g' $_WORKDIR/mdppath/ti_l*mdp
! sed -i 's/delta-lambda.*/delta-lambda = 4e-5/g' $_WORKDIR/mdppath/ti_l0.mdp  # 1/25,000 = 4e-5
! sed -i 's/delta-lambda.*/delta-lambda = -4e-5/g' $_WORKDIR/mdppath/ti_l1.mdp

# Show contents of our working directory.
!ls -l $_WORKDIR

total 12
drwxr-xr-x 3 root root 4096 Nov  8 21:14 lysopath
drwxr-xr-x 2 root root 4096 Nov  8 21:26 mdppath
drwxr-xr-x 4 root root 4096 Nov  8 20:55 struct_top


## 1.&nbsp; Input Files Preparation

In [7]:
#@title 1.1&nbsp; Setup folder structure

# Initialize the free energy environment object. It will store the main 
# parameters for the calculations.
fe = AbsoluteDG(ligList=['lysozyme_benzene'], 
                apoCase='lysozyme_apo', 
                bDSSB=False,
                gmxexec='/usr/local/gromacs/bin/gmx')

# Set the workpath in which simulation input files will be created.
fe.workPath = f'{_WORKDIR}/lysopath'
# Set the path to the MDP files.
fe.mdpPath = f'{_WORKDIR}/mdppath'
# Set the number of replicas.
fe.replicas = 2
# Provide the path to the structures and topologies.
fe.structTopPath = f'{_WORKDIR}/struct_top'

# Prepare the directory structure with all simulations steps required.
fe.simTypes = ['em',  # Energy minimization.
               'eq_posre',  # Equilibrium sim with position restraints.
               'eq',  # Equilibrium simulation.
               'transitions']  # Alchemical, non-equilibrium simulations.

fe.prepareFreeEnergyDir()


---------------------
Summary of the setup:
---------------------

   workpath: /content/pmxrun_matteo/lysopath
   mdp path: /content/pmxrun_matteo/mdppath
   # ligands: 1
   ligands:
        lysozyme_benzene
   apo state: lysozyme_apo

---------------------
Directory structure:
---------------------

/content/pmxrun_matteo/lysopath/
|
|--ligX
|--|--water
|--|--|--stateA
|--|--|--|--run1/2
|--|--|--|--|--/em/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2
|--|--|--|--|--/em/eq_posre/eq/transitions
|--|--protein
|--|--|--stateA
|--|--|--|--run1/2
|--|--|--|--|--/em/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2
|--|--|--|--|--/em/eq_posre/eq/transitions
|--|--strTopFolder
|--lig..

DONE


In [8]:
#@title 1.2&nbsp; Assemble simulation systems

# Assemble the systems: build Gromacs structure and topology for the 
# ligand+water and ligand+protein+water systems.
fe.assemble_systems()

# Define the simulation boxes, fill them with water molecules, and add ions to 
# neutralize the system and reach desired NaCl concentration (0.15 M by default).
fe.boxWaterIons()

----------------------
Assembling the systems
----------------------
Order: ligand-protein-other-water
--- Assembling structures: lysozyme_benzene ---
--- Assembling topologies: lysozyme_benzene ---
----------------
Box, water, ions
----------------


In [9]:
# Check the files present now. This is the content of root dir of the 
# calculation for this protein-ligand pair. The following ls calls show 
# the content of some subfolders for the complex simulations, 
# stateA (coupled ligand). The other folders have the same structure/files but 
# for the ligand in water simulations and for stateB too.
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/"
!ls $_WORKDIR/lysopath/lysozyme_benzene/
!echo ""
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/protein"
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/
!echo ""
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/protein/stateA"
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/stateA/
!echo ""
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/protein/stateA/run1/"
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/stateA/run1/
!echo ""
!echo "> ls ${_WORKDIR}/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre"
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre/

> ls $/content/pmxrun_matteo/lysopath/lysozyme_benzene/
protein  strTopFolder_apo  strTopFolder_holo  water

> ls $/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein
stateA	stateB

> ls $/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA
box.pdb   mdout.mdp  run2	 topol.top  tpr.tpr
ions.pdb  run1	     system.pdb  top.top    water.pdb

> ls $/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run1/
em  eq	eq_posre  transitions

> ls $/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre
ls: cannot access '/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run1/em_posre/': No such file or directory


## 2&nbsp; Energy Minimization

In [10]:
#@title 2.1&nbsp; Prepare TPR files

# Call grompp and create TPR files for all systems and repeats.
# (maldeghi): why EM with -DPOSRES? To be checked.
tpr_files = fe.prepare_simulation(simType='em')

print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

-----------------------------------------
Preparing simulation: em
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run1/em/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run2/em/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run1/em/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run2/em/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run1/em/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run2/em/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateB/run1/em/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateB/run2/em/tpr.tpr


In [11]:
#@title 2.2&nbsp; Run minimizations

# Read the TPR files and run all minimizations.
for tpr_file in tqdm(tpr_files):

  # If minimization has been run already, skip.
  if mdrun_completed(tpr_file):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Run the simulation (with reduced number of steps to speed things up)
  md = mdrun(tpr_file, pmegpu=False, nsteps=1000)

100%|██████████| 8/8 [00:00<00:00, 1361.18it/s]

`/content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run1/em/tpr.tpr` already ran successfully
`/content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run2/em/tpr.tpr` already ran successfully
`/content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run1/em/tpr.tpr` already ran successfully
`/content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run2/em/tpr.tpr` already ran successfully
`/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run1/em/tpr.tpr` already ran successfully
`/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run2/em/tpr.tpr` already ran successfully
`/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateB/run1/em/tpr.tpr` already ran successfully
`/content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateB/run2/em/tpr.tpr` already ran successfully


## 3.&nbsp; Equilibrium Simulations

First, we run a short MD simulation with position restraints to equilibrate the solvent and ions around the protein/ligand. Then, we run tha actual production simulation.

In [12]:
#@title 3.1&nbsp; Prepare TPR files for short equilibration

tpr_files = fe.prepare_simulation(simType='eq_posre', prevSim='em')

print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

-----------------------------------------
Preparing simulation: eq_posre
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run1/eq_posre/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run2/eq_posre/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run1/eq_posre/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run2/eq_posre/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run1/eq_posre/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run2/eq_posre/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateB/run1/eq_posre/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateB/run2/eq_posre/tpr.tpr


In [13]:
#@title 3.2&nbsp; Run short equilibration

# Read the TPR files and run all simulations.
for tpr_file in tqdm(tpr_files):

  # If simulation has been run already, skip.
  if mdrun_completed(tpr_file):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Run the simulation (with reduced number of steps to speed things up)
  md = mdrun(tpr_file, pmegpu=True, nsteps=50_000)  # 50,000 x 2 fs = 100 ps

100%|██████████| 8/8 [16:09<00:00, 121.13s/it]


In [14]:
# Show output files for one run.
print(os.listdir("/".join(tpr_files[0].split("/")[:-1])))
print()

# Tail log file to see performance.
tail(tpr_files[0].replace('tpr.tpr', 'md.log'), n=5)

['md.log', 'dhdl.xvg', 'ener.edr', 'traj.trr', 'confout.gro', 'state.cpt', 'mdout.mdp', 'tpr.tpr']

       Time:       62.133       31.067      200.0
                 (ns/day)    (hour/ns)
Performance:      278.115        0.086
Finished mdrun on rank 0 Tue Nov  8 21:27:28 2022



In [15]:
#@title 3.3&nbsp; Prepare TPR files for equilibrium simulations

tpr_files = fe.prepare_simulation(simType='eq', prevSim='eq_posre')

print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

-----------------------------------------
Preparing simulation: eq
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run1/eq/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run2/eq/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run1/eq/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run2/eq/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run1/eq/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run2/eq/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateB/run1/eq/tpr.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateB/run2/eq/tpr.tpr


In [16]:
#@title 3.4&nbsp; Run equilibrium simulations

# Read the TPR files and run all simulations.
for tpr_file in tqdm(tpr_files):

  # If simulation has been run already, skip.
  if mdrun_completed(tpr_file):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Run the simulation (with reduced number of steps to speed things up)
  md = mdrun(tpr_file, pmegpu=True, nsteps=100_000)  # 100,000 x 2 fs = 200 ps

100%|██████████| 8/8 [32:18<00:00, 242.34s/it]


In [17]:
i = 0

# Show output files for one run.
print(os.listdir("/".join(tpr_files[i].split("/")[:-1])))
print()

# Tail log file to see performance.
tail(tpr_files[i].replace('tpr.tpr', 'md.log'), n=5)

['md.log', 'dhdl.xvg', 'ener.edr', 'traj.trr', 'confout.gro', 'state.cpt', 'mdout.mdp', 'tpr.tpr']

       Time:      120.963       60.482      200.0
                 (ns/day)    (hour/ns)
Performance:      285.709        0.084
Finished mdrun on rank 0 Tue Nov  8 21:49:53 2022



## 4.&nbsp; Non-Equilibrium Simulations

In [101]:
#@title 4.1&nbsp; Extract frames from equilibrium runs, and prepare TPRs

fe.equilTime = 20.  # ps to discard as equilibration
fe.bGenTiTpr = True  # Generates TPRs from extracted frames and rm GRO file.
tpr_files = fe.prepare_simulation(simType='transitions')

print("List of TPR files that we'll run:")
for f in tpr_files:
  print(f"  {f}")

-----------------------------------------
Preparing simulation: transitions
-----------------------------------------
List of TPR files that we'll run:
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run1/transitions/ti1.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run1/transitions/ti2.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run2/transitions/ti1.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateA/run2/transitions/ti2.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run1/transitions/ti1.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run1/transitions/ti2.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run2/transitions/ti1.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/water/stateB/run2/transitions/ti2.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_benzene/protein/stateA/run1/transitions/ti1.tpr
  /content/pmxrun_matteo/lysopath/lysozyme_be

In [102]:
#@title 4.2&nbsp; Run non-equilibrium simulations

# Read the TPR files and run all simulations.
for tpr_file in tqdm(tpr_files):

  # If simulation has been run already, skip.
  if mdrun_completed(tpr_file, transition=True):
    print(f"`{tpr_file}` already ran successfully")
    continue

  # Run the simulation (with reduced number of steps to speed things up)
  md = mdrun(tpr_file, pmegpu=True, transition=True)

100%|██████████| 16/16 [19:03<00:00, 71.45s/it] 


In [104]:
i = 0  # tpr id

# Show output files for one run.
print(os.listdir("/".join(tpr_files[i].split("/")[:-1])))
print()

# Tail dhdl file.
tail("/".join(tpr_files[i].split("/")[:-1]) + "/dhdl_1.xvg", n=10)

['md.log', 'ti1.tpr', '.ipynb_checkpoints', 'dhdl_2.xvg', 'ener.edr', 'ti2.tpr', 'confout.gro', 'state.cpt', 'mdout.mdp', 'dhdl_1.xvg']

49.9820 -19.893085
49.9840 -17.897543
49.9860 -15.790275
49.9880 -14.006278
49.9900 -12.818290
49.9920 -12.357108
49.9940 -12.507529
49.9960 -13.074424
49.9980 -13.965568
50.0000 -15.051780


## 5.&nbsp; Free Energy Estimation

In [129]:
from glob import glob
from pmx.analysis import read_dgdl_files
from pmx.estimators import BAR


def estimate_dg(path_to_water_or_protein, T=298.15, write_dg=True, reverseB=False):
  """Estimate a free energy difference with BAR.

  Args:
    path_to_water_or_protein (str)
    T (float): temperature in Kelvin. Default is 298.15 K.
    write_dg (bool): whether to write the dg estimate to file. Default is True.
    reverseB (bool): whether to reverse the work values for the backward B->A
      transformation. Useful when both forward and reverse simulations were run 
      from lambda zero to one in Gromacs. Default is False.
  """
  
  filesAB = glob(f'{path_to_water_or_protein}/stateA/run*/transitions/dhdl_*.xvg')
  filesBA = glob(f'{path_to_water_or_protein}/stateB/run*/transitions/dhdl_*.xvg')

  # Non-zero values if a sigmoidal lambda interpolation was used.
  sigmoid = 0.

  res_ab = read_dgdl_files(filesAB, lambda0=0, invert_values=reverseB, verbose=False, sigmoid=sigmoid)
  res_ba = read_dgdl_files(filesBA, lambda0=1, invert_values=reverseB, verbose=False, sigmoid=sigmoid)

  bar = BAR(res_ab, res_ba, T=T)
  dg = bar.dg # in kJ/mol

  if write_dg:
    results_file = f'{path_to_water_or_protein}/dg.txt'
    with open(results_file, 'w') as f:
      f.write(f'{dg} kJ/mol')

  return dg


def get_restr_dg(path_to_file):
  with open(path_to_file,'r') as f:
    lines = f.readlines()
  return float((lines[0].rstrip().split()[-2]))

In [130]:
#@title 5.1&nbsp; Get all the relevant terms
dg_water = estimate_dg(f'{_WORKDIR}/lysopath/lysozyme_benzene/water/')
dg_protein = estimate_dg(f'{_WORKDIR}/lysopath/lysozyme_benzene/protein/')
dg_restr = get_restr_dg(f'{_WORKDIR}/lysopath/lysozyme_benzene/strTopFolder_holo/restr_dG.dat')

In [134]:
#@title 5.1&nbsp; Combine all the terms into the binding free energy estimate